<img style="float: left;;" src='Figures/iteso.jpg' width="100" height="200"/></a>

# <center> <font color= #000047> Tratamiento de datos Faltantes </font> </center>

El manejo de datos faltantes es un aspecto fundamental en el análisis de datos. Comprender la naturaleza de los datos faltantes permite seleccionar el método de imputación más adecuado y evitar sesgos en los resultados.


## Visualización de Datos Faltantes

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import pandas as pd

In [ ]:
df=pd.read_csv('API_SI.POV.DDAY_DS2.csv',encoding='latin-1',sep='\t')
df.head(10)

In [ ]:
# Mapa de calor


#### Librería para visualización

In [ ]:
# Instalación de missingno si es necesario
#!pip install missingno

In [ ]:
# Visualización de datos faltantes con missingno


In [ ]:
#Bar plot 
#color='steelblue')
#color="tomato")
#color="tab:green")

## Clasificación de Datos Faltantes

Rubin (1976) propuso una clasificación ampliamente aceptada de los mecanismos de datos faltantes:

> **MCAR (Missing Completely At Random)**: Los datos faltantes ocurren completamente al azar y no dependen ni de los valores observados ni de los no observados.

> **MAR (Missing At Random)**: La probabilidad de que un dato esté ausente depende solo de los valores observados, no de los valores faltantes.

> **MNAR (Missing Not At Random)**: La probabilidad de que un dato esté ausente depende de los valores faltantes en sí mismos, incluso después de considerar los valores observados.
 
<img src="Figures/tipo_faltante.png" width="600" height="600">

[https://towardsdatascience-com.translate.goog/missing-value-imputation-explained-a-visual-guide-with-code-examples-for-beginners-93e0726284eb/?_x_tr_sl=en&_x_tr_tl=es&_x_tr_hl=es&_x_tr_pto=tc]('liga')

### MCAR (Missing Completely at Random)
MCAR es el más sencillo de los tres. Un conjunto de datos es MCAR si la probabilidad de que falte un dato es igual para todos los individuos y no depende de las medidas de otras variables, es decir, no existe ninguna relación entre que un dato sea faltante u observado.

**Ejemplo**:
- Partiendo de una tabla de una base de datos, si ocurre un problema informático y se pierden algunos valores de algunas observaciones de forma aleatoria tendríamos pérdida MCAR.

- En una encuesta, algunas personas omiten una pregunta por accidente, sin relación con sus características o respuestas.

- [Iris (UCI Machine Learning Repository)](https://archive.ics.uci.edu/ml/datasets/iris). Si se eliminan filas aleatoriamente o se simulan valores faltantes en cualquier columna sin relación con las variables, los datos faltantes serían MCAR.

- [Wine Quality (UCI)](https://archive.ics.uci.edu/ml/datasets/wine+quality). Si durante la medición de la calidad del vino, algunos sensores fallan aleatoriamente y se pierden datos de variables químicas sin relación con otras variables o con la calidad.

Para comprobar si un conjunto de datos tiene una pérdida MCAR podemos emplear el [test de Litle](https://bpb-us-w2.wpmucdn.com/blog.nus.edu.sg/dist/4/6502/files/2018/06/mcartest-zlxtj7.pdf)



In [ ]:
# Ejemplo MCAR: Simulación
import numpy as np
import pandas as pd

np.random.seed(42)
data = pd.DataFrame({
    'edad': np.random.randint(20, 60, 100),
    'ingreso': np.random.randint(20000, 80000, 100)
})
# Introducimos valores faltantes aleatoriamente (MCAR)
mask = np.random.rand(*data.shape) < 0.1
data_mcar = data.mask(mask)
print('Datos con valores faltantes MCAR:')
print(data_mcar.head())

#### Prueba de Little para MCAR (Little's MCAR Test)

La prueba de Little (Little's MCAR test) es un método estadístico para evaluar si los datos faltantes son MCAR (Missing Completely At Random). Si el valor p es alto (por ejemplo, > 0.05), no se rechaza la hipótesis nula de que los datos son MCAR.

## MAR (Missing At Random)

Un conjunto de datos tiene pérdida MAR cuando la probabilidad de que una varibale tenga datos faltantes es independiente de los valores de la misma variable, pero dependiente de los valores de otras variables presentes en el conjunto de datos. La probabilidad de que un dato esté ausente depende solo de los valores observados, no de los valores faltantes

**Ejemplo**
- Partiendo de un estudio en el que tenemos las variables puesto de trabajo y sueldo, estaríamos con una pérdida MAR si la gente con determinados puestos de trabajo fuera más reacia a contestar a la pregunta del sueldo.
  
- En un estudio médico, las personas mayores tienden a omitir preguntas sobre ingresos, pero dentro de cada grupo de edad, la omisión es aleatoria.

- [Medical Expenditure Panel Survey (MEPS)](https://meps.ahrq.gov/mepsweb/). La variable 'gastos médicos' puede faltar más frecuentemente en personas jóvenes, pero dentro de cada grupo de edad, la omisión es aleatoria.

- [Titanic (Kaggle)](https://www.kaggle.com/c/titanic/data). En el dataset Titanic, la variable 'Age' tiene valores faltantes. La probabilidad de que falte la edad depende de otras variables observadas como 'Pclass' o 'Sex'. Por ejemplo, es más probable que falte la edad en pasajeros de tercera clase.

Si los datos son MAR, se pueden usar métodos de imputación más sofisticados (como imputación múltiple) para obtener estimaciones no sesgadas.


In [ ]:
# Ejemplo MAR: Simulación
data_mar = data.copy()
# Si la edad es mayor a 50, hay más probabilidad de que ingreso sea NaN
prob = np.where(data_mar['edad'] > 50, 0.4, 0.05)
mask = np.random.rand(len(data_mar)) < prob
data_mar.loc[mask, 'ingreso'] = np.nan
print('Datos con valores faltantes MAR:')
print(data_mar.head(10))

##  MNAR (missing not at random)
Tenemos el caso de MNAR cuando la probabilidad de que una observación sea dato faltante es dependiente del valor de la propia variable. Su efecto no se puede ignorar ya que el valor faltante está relacionado con la razón por la que falta el dato.

**Ejemplo**

- En un estudio acerca de malos hábitos alimenticios, si algunas personas con esos malos hábitos fueran menos propensas a contestar las preguntas.
- En una encuesta de ingresos, las personas con ingresos muy altos tienden a no responder la pregunta sobre ingresos.
- [NHANES (National Health and Nutrition Examination Survey)](https://wwwn.cdc.gov/nchs/nhanes/). En encuestas de salud, las personas con mayor peso pueden ser menos propensas a reportar su peso. Así, los valores faltantes en la variable 'peso' dependen del propio peso (no observado).
- [European Social Survey (ESS)](https://www.europeansocialsurvey.org/). Preguntas sensibles como 'uso de drogas' o 'salud mental' pueden tener faltantes porque quienes tienen valores extremos tienden a no responder.

In [ ]:
# Ejemplo MNAR: Simulación
data_mnar = data.copy()
# Mayor probabilidad de ser NaN si el ingreso es alto
prob = (data_mnar['ingreso'] > 60000).astype(float) * 0.5 + 0.05
mask = np.random.rand(len(data_mnar)) < prob
data_mnar.loc[mask, 'ingreso'] = np.nan
print('Datos con valores faltantes MNAR:')
print(data_mnar.head(10))

#### Resumen Comparativo

| Tipo  | ¿Depende de valores observados? | ¿Depende de valores faltantes? | Ejemplo típico |
|-------|:------------------------------:|:-----------------------------:|:--------------:|
| MCAR  | No                             | No                            | Omisión accidental |
| MAR   | Sí                             | No                            | Omisión por grupo observado |
| MNAR  | Sí/No                          | Sí                            | Omisión por valor oculto |



# Métodos básicos para tratamiento de datos faltantes

## Análisis con datos completos (listwise)
Es un método muy habitual y sencillo de utilizar. Consiste en eliminar todas las observaciones que contengan algún valor faltante en alguna variable del conjunto de datos, es decir, para realizar el análisis estadístico solo se usarían las observaciones que disponen de todos los valores.

**Ejemplo:** 
Partamos de un dataset $X$ con $k = 3$ variables y $n = 100$ observaciones. Una variable tendrá valores faltantes $(p = 1)$ en $m = 10$ observaciones. Si realizamos un análisis con datos completos estaríamos eliminando los datos de las m observaciones con valores faltantes y nuestro dataset pasaría de $n = 100$ observaciones a $n − m = 90$ observaciones.

- Si la pérdida del conjunto de datos es MCAR, los resultados del análisis serán insesgados pues se trataría de una muestra aleatoria de los datos. La desventaja es que no es habitual la presencia de una pérdida MCAR y el análisis sería insesgado en el resto de los casos. 

- Otra desventaja es que se puede perder mucha información, sobre todo si se tienen muchas variables con valores faltantes. Siguiendo con el ejemplo anterior, si tuviéramos un 10% de datos faltantes en cada una de las 3 variables podríamos llegar a eliminar hasta el 30% de las observaciones.

In [ ]:
dfcopy.head() #dataset indicando valores faltantes

In [ ]:
#valores faltantes

In [ ]:
# Definir un threshold de valores faltantes a eliminar por registros y variables


In [ ]:
#Visualización de datos faltantes después de la eliminación


In [ ]:
#Eliminación de datos por registros y variables
# Simulando un Dataset con valores faltantes
df_data=pd.DataFrame(np.random.randn(100,4)+10*np.random.rand(4),columns=['A','B','C','D'])
for c in df_data.columns[:-1]:
    inan=np.random.randint(100,size=np.random.randint(20))
    df_data.loc[inan, c]=np.NaN

In [ ]:
df_data

In [ ]:
# Eliminación de filas (Observaciones)


In [ ]:
# Eliminación de columnas (Variables)


## Métodos de Imputación

Otra forma de tratar los valores faltantes es imputar el valor faltante por un valor. Se usa la información presente en los valores observados para establecer un valor en aquellos valores no observados. Es importante saber elegir bien el método de imputación ya que cada uno tiene sus ventajas e inconvenientes.

- A la hora de hacer la imputación es importante mantener la consistencia de los datos. También es necesario mantener las distribuciones de las variables, así como sus correlaciones para evitar una distorsión de los datos.

### Imputación por un métricas centrales de posición

#### Imputación por la media
Sustituye los valores faltantes de cada variable por la media muestral de la propia variable.

- Este método funciona bien bajo el supuesto de datos MCAR.

-  No funciona bien cuando los valores faltantes dependen de otra variable. Esto es porque si se sustituyen los datos faltantes por la media, estaríamos reduciendo la varianza de cada variable y, por ende, también se modifcarían las matrices de covarianza y de correlaciones.

- Otra desventaja de este procedimiento es que tan solo es aplicable a variables cuantitativas y no a a variables cualitativas.


In [ ]:
# Crear un DataFrame de ejemplo
data = {
    'A': [1, 2, None, 4],
    'B': [5, None, None, 8],
    'C': [9, 10, 11, 12]
}
df = pd.DataFrame(data)
print('DataFrame original:')
print(df)

In [ ]:
# Imputar con la media


In [ ]:
from sklearn.impute import SimpleImputer

# Imputar con la media


In [ ]:
#ver las distribuciones


#### Imputación por la mediana

Se trata de un procedimiento similar al de la media, con la diferencia de que se sustituyen los valores de cada variable por la mediana de la propia variable. 

- **ventaja:** la imputación por la mediana es más robusta a la aparición de datos atípicos. Al igual que la imputación por la media funciona bien bajo el supuesto MCAR.
- Si no es MCAR, estaríamos reduciendo mucho la varianza de cada variable y modifcando las matrices de covarianza y correlaciones. También es aplicable sólo a variables cuantitativas.
- No recomendable si los datos están sesgados.

In [ ]:
# Imputar con la mediana


In [ ]:
# Imputar con la mediana con la librería


In [ ]:
#ver dist


#### Imputación por la moda

Procedimiento muy similar a los dos descritos anteriormente, con la diferencia de que se imputa con el valor más frecuente o moda.

- Se puede usar para imputar las variables cualitativas o categóricas. Por el contrario, no es recomendable usarlo para imputar variables cuantitativas pues la media o mediana son una mejor representación de este tipo de variables. 
- **Desventaja:** Puede reducir la varianza de los datos, no considera la relación entre variables, no recomendable si los datos están sesgados.

In [ ]:
# Imputar con la moda


In [ ]:
# Imputar con la moda


##### Ejemplo

In [ ]:
# otro ejemplo titanic
# Seleccionar columnas categóricas con valores faltantes


#### Imputación Adelante (Forward Fill) y Atrás (Backward Fill)

Consiste en rellenar los valores faltantes con el valor anterior (forward fill) o posterior (backward fill). Efectivos para rellenar valores faltantes en series temporales o datos ordenados.

##### Imputación Adelante (Forward Fill)

Consiste en reemplazar cada valor faltante con el último valor observado previamente en la serie.

Formalmente, para una serie $x_1, x_2, \ldots, x_n$ con valores faltantes, la imputación adelante se define como:

$$
x_t = \begin{cases}
    x_t, & \text{si } x_t \text{ no es faltante} \\
    x_{t-1}^{*}, & \text{si } x_t \text{ es faltante}
\end{cases}
$$

donde $x_{t-1}^{*}$ es el valor imputado más reciente antes de $t$.

##### Imputación Atrás (Backward Fill)

Consiste en reemplazar cada valor faltante con el siguiente valor observado en la serie.

$$
x_t = \begin{cases}
    x_t, & \text{si } x_t \text{ no es faltante} \\
    x_{t+1}^{*}, & \text{si } x_t \text{ es faltante}
\end{cases}
$$

donde $x_{t+1}^{*}$ es el siguiente valor observado después de $t$.

**Desventajas:**
- Puede propagar errores si hay secuencias largas de valores faltantes.
- No recomendable para datos no ordenados.

In [ ]:
# Crear una serie temporal con valores faltantes
np.random.seed(0)
dates = pd.date_range('2023-01-01', periods=20)
values = np.random.randn(20).cumsum()
series = pd.Series(values, index=dates)

# Introducir valores faltantes
series.iloc[[3, 4, 10, 11, 12, 17]] = np.nan
print('Serie original con valores faltantes:')
print(series)


In [ ]:
# Imputación adelante (forward fill)

# Imputación atrás (backward fill)


In [ ]:
# Visualización
plt.figure(figsize=(10,5))
plt.plot(series, 'o-', label='Original')
plt.plot(series_ffill, 's--', label='Forward Fill')
plt.plot(series_bfill, 'd--', label='Backward Fill')
plt.xticks(rotation=30, ha='right')
plt.legend()
plt.title('Imputación Forward Fill y Backward Fill')
plt.xlabel('Fecha')
plt.ylabel('Valor')
plt.show()

- Es recomendable usar estos métodos solo cuando los datos tienen un orden natural (por ejemplo, tiempo).
- Si los valores faltantes están al inicio o final de la serie, forward fill o backward fill pueden no imputar todos los valores.
- Se pueden combinar ambos métodos para imputar valores al inicio y final:

```python
serie_imputada = serie.ffill().bfill()
```

### Imputación Hot Deck y Cold Deck

La imputación Hot Deck y Cold Deck son técnicas utilizadas para rellenar valores faltantes especialmente en encuestas y estudios sociales. Ambas se basan en reemplazar valores faltantes con valores observados de otros registros, pero difieren en la fuente de los datos donantes.

#### Imputación Hot Deck

Consiste en reemplazar cada valor faltante con un valor observado de otro registro dentro del mismo conjunto de datos (el "deck caliente"). El donante puede seleccionarse aleatoriamente, por proximidad, o por pertenencia a un grupo similar.

$$x_{i,\text{imputado}} = x_{j,\text{observado}}, \quad j \in \mathcal{D}, \quad j \neq i$$
donde $\mathcal{D}$ es el conjunto de donantes válidos. Usando este método tenemos la ventaja de que sirve tanto para variables categóricas como numéricas, también que se preserva la distribución de cada variable.

#### Imputación Cold Deck

Similar a Hot Deck, pero los valores donantes provienen de un conjunto de datos externo o de una fuente histórica (el "deck frío").

$$x_{i,\text{imputado}} = x_{k,\text{externo}}, \quad k \in \mathcal{D}_{\text{externo}}$$

donde $\mathcal{D}_{\text{externo}}$ es el conjunto de donantes del dataset externo.

**Hot Deck:** Mantiene la coherencia interna del dataset. Puede preservar la distribución y relaciones entre variables. Puede ser aleatorio o basado en similitud.

**Cold Deck:** Útil cuando se dispone de fuentes externas confiables. Puede introducir sesgos si los datos externos no son comparables.

**Desventajas:**
- Puede ser sensible a la selección de donantes.
- Podría incurrir en sesgos en las estimaciones de las correlaciones de las variables

In [ ]:
# Simulación de datos
np.random.seed(42)
df = pd.DataFrame({
    'edad': np.random.randint(18, 65, 20),
    'ingreso': np.random.randint(10000, 50000, 20)
})
# Introducir valores faltantes
df.loc[[2, 5, 7, 12], 'ingreso'] = np.nan
print('Datos originales con valores faltantes:')
print(df)

In [ ]:
# Imputación Hot Deck aleatoria


In [ ]:
#grupo edad
df['grupo_edad'] = pd.cut(df['edad'], bins=[17, 30, 45, 65], labels=['Joven', 'Adulto', 'Mayor'])
df.head()

In [ ]:
# Introducimos de nuevo valores faltantes para el ejemplo
df.loc[[2, 5, 7, 12], 'ingreso'] = np.nan
df_imp_hotdeck_group = df.copy()
df_imp_hotdeck_group.head(15)

In [ ]:
# Imputación Hot Deck por grupo


In [ ]:
#Imputación Cold Deck usando un Dataset Externo: Supongamos que tenemos un dataset histórico con la misma estructura y lo usamos como fuente de donantes
#Dataset externo (histórico)
df_ext = pd.DataFrame({
    'edad': np.random.randint(18, 65, 20),
    'ingreso': np.random.randint(12000, 48000, 20)
})


In [ ]:
# Imputación Cold Deck: tomar valores de ingreso del dataset externo


## Imputación por Modelos Predictivos

#### Imputación mediante un modelo de regresión o estimación de media condicional

Este método consiste en realizar un modelo de regresión para imputar los valores faltantes con los valores predichos por el modelo de regresión.

Sea $Y$ una variable con valores faltantes, la imputación por regresión se basa en ajustar un modelo de la forma:


$$Y = \beta_0 + \beta_1 X_1 + \beta_2 X_2 + \dots + \beta_p X_p + \varepsilon$$

Donde:
- $Y$ es la variable con valores faltantes.
- $X_1, X_2, \dots, X_p$ son las variables predictoras.
- $\beta_0, \beta_1, \dots, \beta_p$ son los coeficientes estimados.
- $\varepsilon$ es el término de error.

Para imputar, se predice $\hat{Y}$ usando los valores observados de $X$ en las filas con $Y$ faltante.

In [ ]:
#Supongamos un dataset con varias variables predictoras. Imputaremos valores faltantes en 'y' usando regresión múltiple.
from sklearn.linear_model import LinearRegression

# Simulación de datos con múltiples variables
np.random.seed(42)
n = 150
X1 = np.random.normal(5, 2, n)
X2 = np.random.normal(10, 3, n)
X3 = np.random.normal(20, 5, n)
y = 3 + 2*X1 - 1.5*X2 + 0.5*X3 + np.random.normal(0, 2, n)
df_multi = pd.DataFrame({'X1': X1, 'X2': X2, 'X3': X3, 'y': y})
df_multi.head()

In [ ]:
# Introducimos valores faltantes en 'y' (simulando valores faltantes)
mask = np.random.rand(n) < 0.18
df_multi.loc[mask, 'y'] = np.nan
df_multi.head()

In [ ]:
# Separar datos completos e incompletos


In [ ]:
# Ajustar regresión múltiple


In [ ]:
# Imputar valores faltantes


**Desventajas:**
- Si se usan modelos paramétricos de regresión se pueden llegar a producir estimaciones sesgadas. Por ejemplo, si se usa un modelo lineal los valores imputados caerán en una línea recta o en el hiperplano, dependiendo de la cantidad de dimensiones.
- La correlación entre los datos imputados es igual a 1, por lo que las correlaciones estarían sobreestimadas.


Una solución puede ser añadir un resíduo aleatorio a cada valor imputado (**regresión estocástica**). Así conseguimos mantener las correlaciones entre las variables y se podría reducir el sesgo de la imputación por regresión. El residuo aleatorio se suele generar a partir de una distribución normal.

La imputación por **regresión estocástica** añade un término aleatorio al valor imputado, preservando la varianza original de los datos. La ecuación es:

$$\hat{Y}_i = \beta_0 + \beta_1 X_{i1} + \dots + \beta_p X_{ip} + \hat{\varepsilon}_i$$

donde $\hat{\varepsilon}_i$ es un valor aleatorio muestreado del residuo del modelo ajustado.

Esto evita que los valores imputados sean demasiado "perfectos" y subestimen la variabilidad.

In [ ]:
# Simulación de datos
np.random.seed(123)
n = 120
X1 = np.random.normal(8, 2, n)
X2 = np.random.normal(15, 4, n)
y = 5 + 1.2*X1 - 0.8*X2 + np.random.normal(0, 3, n)
df_stoch = pd.DataFrame({'X1': X1, 'X2': X2, 'y': y})

# Introducimos valores faltantes en 'y'
mask = np.random.rand(n) < 0.2
df_stoch.loc[mask, 'y'] = np.nan
df_stoch.head()

In [ ]:
# Separar datos completos e incompletos
df_complete = df_stoch[df_stoch['y'].notnull()]
df_missing = df_stoch[df_stoch['y'].isnull()]


In [ ]:
# Ajustar modelo de regresión
model = LinearRegression()
model.fit(df_complete[['X1', 'X2']], df_complete['y'])

# Calcular residuos del modelo
residuals = df_complete['y'] - model.predict(df_complete[['X1', 'X2']])
residuals

In [ ]:
# Imputar valores faltantes añadiendo ruido aleatorio (regresión estocástica)
imputed_values = model.predict(df_missing[['X1', 'X2']]) + np.random.choice(residuals, size=len(df_missing))
df_stoch.loc[df_stoch['y'].isnull(), 'y'] = imputed_values
df_stoch.head()

###  Imputación de Valores Faltantes por KNN (K-Nearest Neighbors)

El método $KNN$ es un modelo predictivo cuyas aplicaciones incluyen la imputación de datos, ya que es un clasificador de aprendizaje supervisado no paramétrico, que utiliza la proximidad para hacer clasificaciones o predicciones sobre la agrupación de un punto de datos individual. En otras palabras, **se estima el valor perdido como la media (en el caso de las variables numéricas) de los valores de los $k$ vecinos u observaciones más cercanos. Así mismo, para las variables categóricas, se utiliza la clase mayoritaria de entre los k más cercanos.**

El valor de $k$ define cuántos vecinos se verificarán para determinar la clasificación del dato faltante, siendo k directamente proporcional a la generación de sesgo e inversamente proporcional a la varianza. En general se recomienda tener un número impar de k para evitar empates en la clasificación.

Para cada valor faltante, el algoritmo:
1. Calcula la distancia entre la observación incompleta y todas las observaciones completas (usualmente distancia euclidiana).
2. Selecciona los k vecinos más cercanos.
3. Imputa el valor faltante usando la media (para variables numéricas) o la moda (para categóricas) de los vecinos.


**Distancia Euclidiana:**
$$
d(x, y) = \sqrt{\sum_{i=1}^n (x_i - y_i)^2}
$$

**Imputación numérica:**
$$
\hat{x}_{\text{miss}} = \frac{1}{k} \sum_{j=1}^k x_{j, \text{vecino}}
$$

**Imputación categórica:**
- Se utiliza la moda de los vecinos.

Algunas ventajas de este método son: aprovecha la similitud entre observaciones y puede preservar relaciones complejas entre variables.

**Desventajas:** Computacionalmente costoso para grandes datasets. Sensible a la escala de las variables (es recomendable normalizar antes de imputar). El valor de $k$ debe seleccionarse cuidadosamente.

In [ ]:
## Imputación para variables cuantitativas
from sklearn.datasets import load_iris
from sklearn.preprocessing import StandardScaler
from sklearn.impute import KNNImputer
# Cargar datos
iris = load_iris(as_frame=True)
df_iris = iris.data.copy()

# Simular valores faltantes
np.random.seed(1)
mask = np.random.rand(*df_iris.shape) < 0.1
df_iris[mask] = np.nan

# Normalizar antes de imputar
scaler = StandardScaler()
df_scaled = pd.DataFrame(scaler.fit_transform(df_iris), columns=df_iris.columns)

In [ ]:
# Imputación KNN


In [ ]:
## Imputación para variables Categóricas
# Para variables categóricas, se recomienda codificar las categorías numéricamente antes de imputar y luego decodificar tras la imputación.
df_cat = pd.DataFrame({
    'color': ['rojo', 'azul', 'verde', np.nan, 'azul', 'rojo', np.nan],
    'forma': ['circulo', 'cuadro', 'triangulo', 'cuadro', np.nan, 'circulo', 'triangulo']
})
df_cat

In [ ]:
# Codificar categorías


In [ ]:
# Imputar con KNN


In [ ]:
# Decodificar
for col in df_cat.columns:
    cats = ['azul', 'circulo', 'cuadro', 'rojo', 'triangulo', 'verde']
    # Ajustar categorías según columna
    if col == 'color':
        mapping = {0: 'azul', 1: 'rojo', 2: 'verde'}
    else:
        mapping = {0: 'circulo', 1: 'cuadro', 2: 'triangulo'}
    df_cat_imputed[col] = df_cat_imputed[col].round().astype(int).map(mapping)

print('Datos categóricos tras imputación KNN:')
df_cat_imputed

In [ ]:
df_cat

### Imputación de Valores Faltantes con Missing Forest (MissForest)

La imputación por "Missing Forest" (MissForest) es un método basado en bosques aleatorios (Random Forests) para imputar valores faltantes en datasets tanto numéricos como categóricos. Es un método no paramétrico y robusto que puede capturar relaciones no lineales y complejas entre variables.

**MissForest** utiliza un enfoque iterativo:
1. Inicializa los valores faltantes (por ejemplo, con la media o moda).
2. Para cada variable con valores faltantes, entrena un Random Forest usando las otras variables como predictores.
3. Imputa los valores faltantes usando las predicciones del modelo.
4. Repite el proceso para todas las variables con valores faltantes hasta que la imputación converge o se alcanza un número máximo de iteraciones.

Para una variable $X_j$ con valores faltantes:

$$X_{j,\text{miss}} = f_{RF}(X_{-j,\text{obs}})$$

donde $f_{RF}$ es el modelo de Random Forest ajustado usando las otras variables $X_{-j}$ como predictores y los valores observados de $X_j$ como objetivo.

El proceso se repite para cada variable con valores faltantes y se actualizan las imputaciones en cada iteración.

Puede manejar variables numéricas y categóricas. Captura relaciones no lineales y complejas. No requiere supuestos de distribución.

**Desventajas:** Computacionalmente intensivo para grandes datasets. Puede sobreajustar si hay pocos datos o muchas variables irrelevantes.

In [ ]:
# Imputación tipo MissForest manual para variables numéricas usando scikit-learn
import numpy as np
import pandas as pd
from sklearn.ensemble import RandomForestRegressor

# Simulación de datos
np.random.seed(0)
df = pd.DataFrame({
    'A': np.random.normal(10, 2, 20),
    'B': np.random.normal(5, 1, 20)
})
df.loc[[2, 5, 7], 'A'] = np.nan
df.loc[[1, 6, 12], 'B'] = np.nan

print('Datos originales con valores faltantes:')
df.head()


In [ ]:
# Imputación iterativa tipo MissForest (solo numérico, ejemplo básico)
df_imputed = df.copy()
for col in df.columns:
    mask = df_imputed[col].isnull()
    if mask.any():
        train = df_imputed.loc[~mask]
        test = df_imputed.loc[mask]
        X_train = train.drop(columns=[col])
        y_train = train[col]
        X_test = test.drop(columns=[col])
        # Imputar valores faltantes en predictores con la media temporalmente
        X_train = X_train.fillna(X_train.mean())
        X_test = X_test.fillna(X_train.mean())
        rf = RandomForestRegressor(n_estimators=100, random_state=0)
        rf.fit(X_train, y_train)
        df_imputed.loc[mask, col] = rf.predict(X_test)

print('\nDatos tras imputación tipo MissForest manual:')
df_imputed.head()

### Imputación múltiple
La imputación múltiple se caracteriza por devolver más de un valor para cada valor faltante. Cada uno de los valores faltantes se imputan m veces, obteniendo m conjuntos de datos completos. Estos múltiples valores se combinan para obtener los valores imputados. Para combinar estos valores se puede usar la media o mediana en el caso de variables numéricas, mientras que para variables categóricas podemos emplear la moda. También se podría escoger
uno de los valores de forma aleatoria.

El proceso de imputación múltiple típicamente sigue estos pasos:
1. **Imputación:** Se generan $m$ datasets completos, imputando los valores faltantes de manera diferente en cada uno (usando métodos estocásticos).
2. **Análisis:** Se realiza el análisis estadístico deseado en cada dataset imputado.
3. **Combinación:** Se combinan los resultados de los $m$ análisis para obtener estimaciones finales y errores estándar ajustados.

Sea $\hat{Q}_i$ la estimación del parámetro de interés en el dataset imputado $i$ ($i=1,\ldots,m$), y $U_i$ su varianza estimada.

- **Estimación combinada:**
$$
\bar{Q} = \frac{1}{m} \sum_{i=1}^m \hat{Q}_i
$$

- **Varianza total:**
$$
T = \bar{U} + \left(1 + \frac{1}{m}\right)B
$$
donde:
$$
\bar{U} = \frac{1}{m} \sum_{i=1}^m U_i \quad \text{y} \quad B = \frac{1}{m-1} \sum_{i=1}^m (\hat{Q}_i - \bar{Q})^2
$$

Al generar varias imputaciones por cada valor faltante y combinarlas, estamos realizando estimaciones más precisas y menos sesgadas de los valores faltantes. Otra ventaja es su posible aplicación tanto en variables numéricas como variables categóricas. Sus desventajas podrían ser el coste computacional de realizar varias imputaciones para cada valor faltante y el hecho de elegir un criterio para combinar estos valores.



In [ ]:
from sklearn.experimental import enable_iterative_imputer  # noqa
from sklearn.impute import IterativeImputer

# Simulación de datos con valores faltantes
np.random.seed(0)
df = pd.DataFrame({
    'A': np.random.normal(10, 2, 20),
    'B': np.random.normal(5, 1, 20),
    'C': np.random.normal(0, 1, 20)
})
df.loc[[2, 5, 7], 'A'] = np.nan
df.loc[[1, 6, 12], 'B'] = np.nan
df.loc[[3, 8, 13], 'C'] = np.nan

print('Datos originales con valores faltantes:')
df


In [ ]:
# Imputación múltiple (MICE) - se puede repetir varias veces para obtener diferentes imputaciones


In [ ]:
# Calcular la media y varianza de 'A' en cada dataset imputado
means = np.array([d['A'].mean() for d in imputed_datasets])
vars_ = np.array([d['A'].var(ddof=1)/len(d) for d in imputed_datasets])  # varianza de la media

In [ ]:
# Estimación combinada y varianza total
Q_bar = means.mean()
U_bar = vars_.mean()
B = means.var(ddof=1)
T = U_bar + (1 + 1/len(means)) * B
std_error = np.sqrt(T)

print(f"Media combinada de 'A': {Q_bar:.3f}")
print(f"Error estándar combinado: {std_error:.3f}")

In [ ]:
#OTRO EJEMPLO
from sklearn.datasets import load_iris

# Cargar datos
iris = load_iris(as_frame=True)
df_iris = iris.data.copy()

# Simular valores faltantes
np.random.seed(1)
mask = np.random.rand(*df_iris.shape) < 0.1
df_iris[mask] = np.nan
df_iris.head()

In [ ]:
# Imputación múltiple (MICE)
imputer = IterativeImputer(random_state=0, sample_posterior=True, max_iter=10)
imputed_iris = []
for i in range(5):
    imputer.random_state = i
    imputed = pd.DataFrame(imputer.fit_transform(df_iris), columns=df_iris.columns)
    imputed_iris.append(imputed)

print('Primeras filas de la primera imputación en Iris:')
print(imputed_iris[0].head())

In [ ]:
#Otro ejemplo
file_path = "https://raw.githubusercontent.com/selva86/datasets/master/Churn_Modelling_m.csv"
df = pd.read_csv(file_path)
df.head()

In [ ]:
df.info()

In [ ]:
df.isnull().sum()

In [ ]:
df_train = df.loc[:, ["Balance", "Age", "Exited"]] # Usamos tres características numéricas
df_train.head()

In [ ]:
imputer.fit(df_train)

In [ ]:
df_imputed = imputer.transform(df_train)

In [ ]:
df_imputed

In [ ]:
df.loc[:, ["Balance", "Age", "Exited"]] = df_imputed
df.head(10)

In [ ]:
df.isnull().sum()

# <center> <font color= #000047> Tratamiento de datos Faltantes Práctica</font> </center>


## Ejemplo1

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import missingno as msno
from sklearn.impute import SimpleImputer, KNNImputer
from sklearn.experimental import enable_iterative_imputer  # noqa
from sklearn.impute import IterativeImputer

In [ ]:
# 1. Cargar un dataset real (Titanic)


In [ ]:
# 2. Análisis exploratorio inicial


In [ ]:
# Seleccionar variables relevantes para imputación (numéricas y categóricas)


In [ ]:
# Visualizar distribución de variables numéricas antes de imputar


In [ ]:
# 3. Imputación de variables numéricas

# a) Imputación con la mediana (robusta a outliers)


In [ ]:
# b) Imputación con KNN (considerando todas las numéricas seleccionadas)


In [ ]:
# c) Imputación con IterativeImputer (MICE)


In [ ]:
# Comparar distribuciones tras imputación para 'age' y 'fare'


## Ejemplo 2:

Diamonds Dataset, contiene información detallada sobre **más de 53,000 diamantes**, incluyendo características físicas, calidad y precio. Cada fila representa un diamante individual.

Features:
- **carat**: Peso del diamante (en quilates).
- **cut**: Calidad del corte (Fair, Good, Very Good, Premium, Ideal).
- **color**: Color del diamante, de J (peor) a D (mejor).
- **clarity**: Pureza del diamante (de I1, SI2, SI1, VS2, VS1, VVS2, VVS1, IF).
- **depth**: Profundidad total como porcentaje del ancho promedio.
- **table**: Ancho de la parte superior del diamante como porcentaje del ancho promedio.
- **price**: Precio en dólares estadounidenses.
- **x**: Longitud en milímetros.
- **y**: Ancho en milímetros.
- **z**: Profundidad en milímetros.

In [ ]:
# Imputación con la media (numéricas)


In [ ]:
# Imputación con la mediana (numéricas)


In [ ]:
# Imputación con KNN (numéricas)


In [ ]:
# Imputación con IterativeImputer (MICE, numéricas)


In [ ]:
# Imputación forward fill y backward fill (numéricas)


In [ ]:
# Imputación aleatoria (numéricas)


In [ ]:
# Visualización comparativa de imputaciones para 'carat'


In [ ]:
# Imputación por la moda (categóricas)


No existe una forma perfecta de tratar los valores perdidos. Cada estrategia puede funcionar mejor para ciertos conjuntos de datos y tipos de datos faltantes, pero puede funcionar mucho peor en otros tipos de conjuntos de datos. 

Hay algunas reglas establecidas para decidir qué estrategia usar para tipos particulares de valores perdidos, pero más allá de eso, se debe experimentar y verificar qué modelo funciona mejor para su conjunto de datos.


### Bibliografía
- Rubin, D. B. (1976). Inference and missing data. *Biometrika*, 63(3), 581-592.
- Little, R. J. A., & Rubin, D. B. (2019). *Statistical Analysis with Missing Data* (3rd ed.). Wiley.
- Schafer, J. L., & Graham, J. W. (2002). Missing data: our view of the state of the art. *Psychological Methods*, 7(2), 147–177.
- Enders, C. K. (2010). *Applied Missing Data Analysis*. Guilford Press.
- Van Buuren, S. (2018). *Flexible Imputation of Missing Data* (2nd ed.). CRC Press.